# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING

Variabili utili per testare il funzionamento


In [6]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


In [7]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

In [21]:
function crossingTest(new::Int, old::Int, count::T, status::Int)::Number where T <: Real
    if status == 0
        status = new
        count += 0.5
    else
        if status == old
        	count += 0.5
        else
        	count -= 0.5
        end
        status = 0
    end
end


crossingTest (generic function with 1 method)

In [4]:
@btime crossingTest(0, 0, 0.5, 0)   #0,041 μs


  0.041 ns (0 allocations: 0 bytes)


1.0

In [20]:
function setTile(box)
	tiles = [[9,1,5],[8,0,4],[10,2,6]]
	b1,b2,b3,b4 = box
	function tileCode(point)
		x,y = point
		code = 0
        if y>b1 code=code|1 end
		if y<b2 code=code|2 end
		if x>b3 code=code|4 end
		if x<b4 code=code|8 end
		return code
	end
	return tileCode
end

setTile (generic function with 1 method)

In [6]:
@btime setTile([0., 0., 0., 0.])   #197,120 μs

  197.120 ns (5 allocations: 416 bytes)


(::var"#tileCode#9"{Float64, Float64, Float64, Float64}) (generic function with 1 method)

## Versione iniziale di pointInPolygonClassification

In [8]:
function pointInPolygonClassification(V,EV)
    function pointInPolygonClassification0(pnt)
        x,y = pnt
        xmin,xmax,ymin,ymax = x,x,y,y
        tilecode = setTile([ymax,ymin,xmax,xmin])
        count,status = 0,0

        for (k,edge) in enumerate(EV)
            p1,p2 = V[:,edge[1]],V[:,edge[2]]
            (x1,y1),(x2,y2) = p1,p2
            c1,c2 = tilecode(p1),tilecode(p2)
            c_edge, c_un, c_int = c1⊻c2, c1|c2, c1&c2

            if (c_edge == 0) & (c_un == 0) return "p_on"
            elseif (c_edge == 12) & (c_un == c_edge) return "p_on"
            elseif c_edge == 3
                if c_int == 0 return "p_on"
                elseif c_int == 4 count += 1 end
            elseif c_edge == 15
                x_int = ((y-y2)*(x1-x2)/(y1-y2))+x2
                if x_int > x count += 1
                elseif x_int == x return "p_on" end
            elseif (c_edge == 13) & ((c1==4) | (c2==4))
                    crossingTest(1,2,status,count)
            elseif (c_edge == 14) & ((c1==4) | (c2==4))
                    crossingTest(2,1,status,count)
            elseif c_edge == 7 count += 1
            elseif c_edge == 11 count = count
            elseif c_edge == 1
                if c_int == 0 return "p_on"
                elseif c_int == 4 crossingTest(1,2,status,count) end
            elseif c_edge == 2
                if c_int == 0 return "p_on"
                elseif c_int == 4 crossingTest(2,1,status,count) end
            elseif (c_edge == 4) & (c_un == c_edge) return "p_on"
            elseif (c_edge == 8) & (c_un == c_edge) return "p_on"
            elseif c_edge == 5
                if (c1==0) | (c2==0) return "p_on"
                else crossingTest(1,2,status,count) end
            elseif c_edge == 6
                if (c1==0) | (c2==0) return "p_on"
                else crossingTest(2,1,status,count) end
            elseif (c_edge == 9) & ((c1==0) | (c2==0)) return "p_on"
            elseif (c_edge == 10) & ((c1==0) | (c2==0)) return "p_on"
            end
        end
        if (round(count)%2)==1
        	return "p_in"
        else
        	return "p_out"
        end
    end
    return pointInPolygonClassification0
end

pointInPolygonClassification (generic function with 1 method)

In [9]:
@btime pointInPolygonClassification(V, EV)   #120,546 μs

  119.650 ns (1 allocation: 32 bytes)


(::var"#pointInPolygonClassification0#11"{Matrix{Float64}, Vector{Vector{Int64}}}) (generic function with 1 method)

## Versione ottimizzata di pointInPolygonClassification

In [10]:
function edgecode1(c_int) #c_edge == 1
    if c_int == 0 return "p_on"
    elseif c_int == 4 crossingTest(1,2,status, counter) end 
end 

edgecode1 (generic function with 1 method)

In [11]:
function edgecode2(c_int) #c_edge == 2
    if c_int == 0 return "p_on"
    elseif c_int == 4 crossingTest(2,1,status, counter) end 
end 

edgecode2 (generic function with 1 method)

In [12]:
function edgecode3(c_int) #c_edge == 3
    if c_int == 0 return "p_on"
    elseif c_int == 4 counter += 1 end 
end 

edgecode3 (generic function with 1 method)

In [13]:
function edgecode4(c_un) #c_edge == 4
    if c_un == 3 return "p_on" end 
end 

edgecode4 (generic function with 1 method)

In [14]:
function edgecode5(c1,c2) #c_edge == 5
    if (c1==0) | (c2==0) return "p_on"
    else crossingTest(1,2,status, counter) end 
end 

edgecode5 (generic function with 1 method)

In [15]:
function edgecode6(c1,c2) #c_edge == 6
    if ((c1==0) | (c2==0)) return "p_on"
    else crossingTest(2,1,status, counter) end 
end 

edgecode6 (generic function with 1 method)

In [16]:
function edgecode7(counter) #c_edge == 7
    counter += 1
end

edgecode7 (generic function with 1 method)

In [17]:
function edgecode8(c_un) #c_edge == 8 (c_un == 6 su slide)
    if (c_un == 8) return "p_on" end   
end

edgecode8 (generic function with 1 method)

In [18]:
function edgecode9_10(c1,c2) #c_edge == 9/10
    if ((c1 ==0) | (c2==0)) return "p_on" end
end

edgecode9_10 (generic function with 1 method)

In [19]:
function edgecode11() #c_edge == 11
end

edgecode11 (generic function with 1 method)

In [20]:
function edgecode12(c_un) #c_edge = 12
    if (c_un == 12 ) return "p_on" end    
end

edgecode12 (generic function with 1 method)

In [21]:
function edgecode13(c1,c2) #c_edge = 13
    if (( c1 ==4) | (c2 == 4))
        crossingTest(1,2,status, counter) end
end

edgecode13 (generic function with 1 method)

In [22]:
function edgecode14(c1,c2) #c_edge = 14
    if (( c1 ==4) | (c2 == 4))
        crossingTest(2,1,status, counter) end
end

edgecode14 (generic function with 1 method)

In [23]:
function edgecode15(x1,x2,y1,y2,x,y)
    x_int = ((y-y2)*(x1-x2)/(y1-y2))+x2
    if x_int > x counter+=1
    elseif (x_int == x) return "p_on" end
end

edgecode15 (generic function with 1 method)

## Versione pointInPolygonClassificationMOD con funzioni separate

In [19]:
function pointInPolygonClassificationMOD(V,EV) 
    function pointInPolygonClassification0(pnt)
        x,y = pnt
        xmin,xmax,ymin,ymax = x,x,y,y
        tilecode = setTile([ymax,ymin,xmax,xmin])
        count,status = 0,0

        for (k,edge) in enumerate(EV)
            p1,p2 = V[:,edge[1]],V[:,edge[2]]
            (x1,y1),(x2,y2) = p1,p2
            c1,c2 = tilecode(p1),tilecode(p2)
            c_edge, c_un, c_int = c1⊻c2, c1|c2, c1&c2

            if (c_edge == 0) & (c_un == 0) return "p_on"
            elseif (c_edge == 12) & (c_un == c_edge) return "p_on"
            elseif c_edge == 3
                edgecode3(c_int)
            elseif c_edge == 15
               edgecode15(x1,x2,y1,y2,x,y)
            elseif (c_edge == 13) & ((c1==4) | (c2==4))
                    crossingTest(1,2,status,count)
            elseif (c_edge == 14) & ((c1==4) | (c2==4))
                    crossingTest(2,1,status,count)
            elseif c_edge == 7 edgecode7(counter)
            elseif c_edge == 11 count = count
            elseif c_edge == 1
               edgecode1(c_int)
            elseif c_edge == 2
                edgecode2(c_int)
            elseif (c_edge == 4) & (c_un == c_edge) return "p_on"
            elseif (c_edge == 8) & (c_un == c_edge) return "p_on"
            elseif c_edge == 5
                edgecode5(c1,c2)
            elseif c_edge == 6
                edgecode6(c1,c2)
            elseif (c_edge == 9) & ((c1==0) | (c2==0)) return "p_on"
            elseif (c_edge == 10) & ((c1==0) | (c2==0)) return "p_on"
            end
        end
        if (round(count)%2)==1
        	return "p_in"
        else
        	return "p_out"
        end
    end
    return pointInPolygonClassification0
end

pointInPolygonClassificationMOD (generic function with 1 method)

In [25]:
@btime pointInPolygonClassificationMOD(V,EV)  #119,540 μs

  119.658 ns (1 allocation: 32 bytes)


(::var"#pointInPolygonClassification0#12"{Matrix{Float64}, Vector{Vector{Int64}}}) (generic function with 1 method)

## Versione iniziale di boundingbox

La funzione boundingbox serve a creare il bounding Box di una cella, cioè la scatola di misura più piccola (area, volume, ipervolume) entro cui sono contenuti tutti i punti.

In [12]:
function boundingbox(vertices::Lar.Points)
   minimum = mapslices(x->min(x...), vertices, dims=2)
   maximum = mapslices(x->max(x...), vertices, dims=2)
   return minimum, maximum
end

boundingbox (generic function with 1 method)

In [27]:
@btime boundingbox(V)   #14,076 μs

  15.190 μs (147 allocations: 5.03 KiB)


([0.0; 0.0;;], [2.0; 1.0;;])

### versione modificata boundingbox

In [18]:
function boundingboxMOD(vertices::Lar.Points)
    firstDim = vertices[1,:]
    secondDim = vertices[2,:]
    if (size(vertices,1)==3)
        thirdDim = vertices[3,:]
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...), min(thirdDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...), max(thirdDim...)])
    else
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...)])
    end
    return fetch(minimum),fetch(maximum)
 end

boundingboxMOD (generic function with 1 method)

In [29]:
@btime boundingboxMOD(V)   #14,076 μs

  20.405 μs (63 allocations: 2.19 KiB)


([0.0; 0.0;;], [2.0; 1.0;;])

## Input_collection

In [17]:
function input_collection(data::Array)::Lar.LAR
	assembly = Lar.Struct(data)
	return Lar.struct2lar(assembly)
end


input_collection (generic function with 1 method)

## Versione iniziale di coordintervals

coordintervals crea un dizionario ordinato dove la chiave è l'intervallo su una coordinata, e come valore associato ha l'indice dell'intervallo corrispondente nel boundig box

In [16]:
function coordintervals(coord,bboxes)
	boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
	for (h,box) in enumerate(bboxes)
		key = box[coord,:]
		if haskey(boxdict,key) == false
			boxdict[key] = [h]
		else
			push!(boxdict[key], h)
		end
	end
	return boxdict
end

coordintervals (generic function with 1 method)

In [32]:
@btime coordintervals(1, bb)   #1,624 μs


  1.135 μs (17 allocations: 1.28 KiB)


OrderedDict{Vector{Float64}, Vector{Int64}} with 4 entries:
  [0.0, 1.0] => [1, 3]
  [1.0, 1.0] => [2]
  [0.0, 0.0] => [4]
  [0.0, 2.0] => [5]

## Versione iniziale di boxcovering

boxcovering calcola quali bounding box si intersecano tra loro.

In [15]:
function boxcovering(bboxes, index, tree)
	covers = [[] for k=1:length(bboxes)]
	for (i,boundingbox) in enumerate(bboxes)
		extent = bboxes[i][index,:]
		iterator = IntervalTrees.intersect(tree, tuple(extent...))
		for x in iterator
			append!(covers[i],x.value)
		end
	end
	return covers
end

boxcovering (generic function with 1 method)

In [97]:
@btime boxcovering(bb, 1, t)   #4,546 μs

  4.546 μs (82 allocations: 3.02 KiB)


5-element Vector{Vector{Any}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

boxcovering è type stable ma la variabile covers è un array di Any. Si procede tipizzando covers e dividendo la funzione in microtask.

### Versione modificata di boxcovering

In [46]:
function boxcoveringMOD(bboxes, index, tree)
    covers = [[zero(eltype(Int64))] for k=1:length(bboxes)]		#zero(eltype(Int64)) serve per rendere covers type stable
    @threads for (i,boundingbox) in collect(enumerate(bboxes))
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        addIntersection(covers, i, iterator)
    end
    return covers
end

boxcoveringMOD (generic function with 1 method)

In [47]:
cov = boxcoveringMOD(bb, 1, t)

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

In [48]:
ex = bb[1][1,:]

2-element Vector{Float64}:
 0.0
 1.0

In [49]:
it = IntervalTrees.intersect(t, tuple(ex...))


IntervalTrees.IntervalIntersectionIterator{typeof(IntervalTrees.true_cmp), Float64, IntervalValue{Float64, Array}, 64}(IntervalTrees.true_cmp, IntervalTrees.Intersection{Float64, IntervalValue{Float64, Array}, 64}(0, #undef), IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}, Interval{Float64}
(0.0,1.0))


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]
(1.0,1.0) => [2]

In [50]:
@btime boxcoveringMOD(bb, 1, t)    #2.8 μs

  9.428 μs (86 allocations: 4.67 KiB)


5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Versione iniziale di spaceindex

spaceindex, dato un modello, restituisce un array di array dove l'elemento i-esimo rappresenta quali intersezioni ha il bounding box i-esimo con gli altri bounding box

In [8]:
function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
	V,CV = model[1:2]
	dim = size(V,1)
	cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
	#----------------------------------------------------------
	bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
	xboxdict = Lar.coordintervals(1,bboxes)
	yboxdict = Lar.coordintervals(2,bboxes)
	# xs,ys are IntervalTree type
	xs = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in xboxdict
		xs[tuple(key...)] = boxset
	end
	ys = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in yboxdict
		ys[tuple(key...)] = boxset
	end
	xcovers = Lar.boxcovering(bboxes, 1, xs)
	ycovers = Lar.boxcovering(bboxes, 2, ys)
	covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]

	if dim == 3
		zboxdict = Lar.coordintervals(3,bboxes)
		zs = IntervalTrees.IntervalMap{Float64, Array}()
		for (key,boxset) in zboxdict
			zs[tuple(key...)] = boxset
		end
		zcovers = Lar.boxcovering(bboxes, 3, zs)
		covers = [intersect(pair...) for pair in zip(zcovers,covers)]
	end
	# remove each cell from its cover
	for k=1:length(covers)
		covers[k] = setdiff(covers[k],[k])
	end
	return covers
end


spaceindex (generic function with 1 method)

In [38]:
@btime spaceindex((V,EV))       #104,214 μs

  108.574 μs (973 allocations: 50.25 KiB)


5-element Vector{Vector{Int64}}:
 [4, 5, 2]
 [1, 3, 5]
 [4, 5, 2]
 [1, 3, 5]
 [4, 1, 3, 2]

### Versione modificata di SpaceIndex

In [43]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees

function spaceindexMOD(model::Lar.LAR)::Array{Array{Int,1},1}
    V,CV = model[1:2]
    dim = size(V,1)
    
    cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]		    #calcola le celle
    bboxes = [hcat(boundingboxMOD(cell)...) for cell in cellpoints]    #calcola i boundingbox delle celle
    
    xboxdict = Threads.@spawn coordintervals(1,bboxes)
    yboxdict = Threads.@spawn coordintervals(2,bboxes)

    # xs,ys sono di tipo IntervalTree
    xs = Threads.@spawn createIntervalTree(fetch(xboxdict))
    ys = Threads.@spawn createIntervalTree(fetch(yboxdict))
    
    xcovers = Threads.@spawn boxcoveringMOD(bboxes, 1, fetch(xs))                        #lista delle intersezioni dei bb sulla coordinata x
    ycovers = Threads.@spawn boxcoveringMOD(bboxes, 2, fetch(ys))                        #lista delle intersezioni dei bb sulla coordinata x
    covers = [intersect(pair...) for pair in zip(fetch(xcovers),fetch(ycovers))]      #lista delle intersezioni dei bb su entrambe le coordinate

    if dim == 3
        zboxdict = Threads.@spawn coordintervals(3,bboxes)
        zs = Threads.@spawn createIntervalTree(fetch(zboxdict))
        zcovers = Threads.@spawn boxcoveringMOD(bboxes, 3, fetch(zs))
        covers = [intersect(pair...) for pair in zip(fetch(zcovers),covers)]
    end
    
    removeIntersection(covers)       #rimozione delle intersezioni con se stesso
    return covers
end

spaceindexMOD (generic function with 1 method)

In [51]:
@btime spaceindexMOD((V, EV))       #101,009 μs

  157.396 μs (538 allocations: 37.27 KiB)


5-element Vector{Vector{Int64}}:
 [4, 5, 2]
 [1, 3, 5]
 [4, 5, 2]
 [1, 3, 5]
 [4, 1, 3, 2]

Dal code_warntype di spaceIndex emerge l'instabilità riguardo alcune variabili e non dell'intero metodo. Si procede nell'analisi delle singole funzioni mono-tasks.

## Versione iniziale di Intersection

In [100]:
function intersection(line1,line2)
	x1,y1,x2,y2 = vcat(line1...)
	x3,y3,x4,y4 = vcat(line2...)

	# intersect lines e1,e2
	# http://www.cs.swan.ac.uk/~cssimon/line_intersection.html
	det = (x4-x3)*(y1-y2)-(x1-x2)*(y4-y3)
	if det != 0.0
		a = 1/det
		b = [y1-y2 x2-x1; y3-y4 x4-x3]  # x1-x2 => x2-x1 bug in the source link !!
		c = [x1-x3; y1-y3]
		(β,α) = a * b * c
	else
		if (y1==y2) == (y3==y4) || (x1==x2) == (x3==x4) # segments collinear
			 return nothing
		else
			 # segments parallel: no intersection
			 return nothing
		end
	end
	return α,β
end


intersection (generic function with 1 method)

In [110]:
@btime intersection([0., 0., 0., 0.], [0., 0., 0., 0.])       #748,214 μs

  748.214 ns (27 allocations: 752 bytes)


## Versione iniziale di linefragments

In [14]:
function linefragments(V,EV,Sigma)
	# remove the double intersections by ordering Sigma
	m = length(Sigma)
	sigma = map(sort,Sigma)
	reducedsigma = sigma ##[filter(x->(x > k), sigma[k]) for k=1:m]
	# pairwise parametric intersection
	params = Array{Float64,1}[[] for i=1:m]
	for h=1:m
		if sigma[h] ≠ []
			line1 = V[:,EV[h]]
			for k in sigma[h]
				line2 = V[:,EV[k]]
				out = Lar.intersection(line1,line2) # TODO: w interval arithmetic
				if out ≠ nothing
					α,β = out
					if 0<=α<=1 && 0<=β<=1
						push!(params[h], α)
						push!(params[k], β)
					end
				end
			end
		end
	end
	# finalize parameters of fragmented lines
	fragparams = []
	for line in params
		push!(line, 0.0, 1.0)
		line = sort(collect(Set(line)))
		push!(fragparams, line)
	end
	return fragparams
end

linefragments (generic function with 1 method)

In [115]:
Sigma = [[4, 5, 2], [1, 3, 5], [4, 5, 2], [1, 3, 5], [4, 1, 3, 2]];
@btime linefragments(V, EV, Sigma)       #33,756 μs

  33.756 μs (800 allocations: 25.34 KiB)


5-element Vector{Any}:
 [0.0, 1.0]
 [0.0, 0.5, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 0.5, 1.0]

## Versione iniziale di fragmentlines

In [21]:
function fragmentlines(model)
	V,EV = model
	# acceleration via spatial index computation
	Sigma = Lar.spaceindex(model)
	# actual parametric intersection of each line with the close ones
	lineparams = Lar.linefragments(V,EV,Sigma)
	# initialization of local data structures
	vertdict = OrderedDict{Array{Float64,1},Array{Int,1}}()
	pairs = collect(zip(lineparams, [V[:,e] for e in EV]))
	vertdict = OrderedDict{Array{Float64,1},Int}()
	W = Array[]
	EW = Array[]
	k = 0
	# generation of intersection points
	for (params,linepoints) in pairs
		v1 = linepoints[:,1]
		v2 = linepoints[:,2]
		points = [ v1 + t*(v2 - v1) for t in params]   # !!!! loved !!
		vs = zeros(Int64,1,length(points))
		PRECISION = 8
		# identification via dictionary of points
		for (h,point) in enumerate(points)
			point = map(Lar.approxVal(PRECISION), point)
			if haskey(vertdict, point) == false
				k += 1
				vertdict[point] = k
				push!(W, point)
			end
			vs[h] = vertdict[point]
		end
		[push!(EW, [vs[k], vs[k+1]]) for k=1:length(vs)-1]
	end
	# normalization of output
	W,EW = hcat(W...),convert(Array{Array{Int64,1},1},EW)
	V,EV = Lar.congruence((W,EW))
	return V,EV
end

fragmentlines (generic function with 1 method)

In [122]:
model = V, EV;
@btime fragmentlines(model)       #229,333 μs

  231.118 μs (2333 allocations: 97.80 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 5], [5, 1], [1, 3], [3, 6]])

## Versione iniziale di fraglines


In [16]:
function fraglines(sx::Float64=1.2,sy::Float64=1.2,sz::Float64=1.2)
	function fraglines0(model)
		V,EV = Lar.fragmentlines(model)

		W = zeros(Float64, size(V,1), 2*length(EV))
		EW = Array{Array{Int64,1},1}()
		for (k,(v1,v2)) in enumerate(EV)
			if size(V,1)==2
				x,y = (V[:,v1] + V[:,v2]) ./ 2
				scx,scy = x*sx, y*sy
				t = [scx-x, scy-y]
			elseif size(V,1)==3
				x,y,z = (V[:,v1] + V[:,v2]) ./ 2
				scx,scy,scz = x*sx, y*sy, z*sz
				t = [scx-x, scy-y, scz-z]
			end
			W[:,2*k-1] = V[:,v1] + t
			W[:,2*k] = V[:,v2] + t
			push!(EW, [2*k-1, 2*k])
		end
		return W,EW
	end
	return fraglines0
end

fraglines (generic function with 4 methods)

In [17]:
@btime fraglines(0., 0., 0.)       #0,041 μs

  0.043 ns (0 allocations: 0 bytes)


(::var"#fraglines0#17"{Float64, Float64, Float64}) (generic function with 1 method)

## Versione iniziale di congruence

In [54]:
function congruence(model)
	W,EW = model
	# congruent vertices
	balltree = NearestNeighbors.BallTree(W)
	r = 0.0000000001
	near = Array{Any}(undef, size(W,2))
@sync	for k=1:size(W,2)
@views	near[k] = union([NearestNeighbors.inrange(balltree, W[:,k], r, true)]...)
	end
	near = map(sort,near)  # check !!!
	for k=1:size(W,2)
		W[:,k] = W[:,near[k][1]]
	end
	pointidx = [ near[k][1] for k=1:size(W,2) ]  # check !!
	invidx = OrderedDict(zip(1:length(pointidx), pointidx))
	V = [W[:,k] for k=1:length(pointidx)]
	# congruent edges
	EV = []
@async	for e in (EW)
@views		newedge = [invidx[e[1]],invidx[e[2]]]
		if newedge[1] !== newedge[2]
			push!(EV,newedge)
		end
	end
	EV = [EV[h] for h=1:length(EV) if length(EV[h])==2]
	EV = convert(Lar.Cells, EV)
	#W,EW = Lar.simplifyCells(V,EV)
	return hcat(V...),EV
end




congruence (generic function with 1 method)

In [55]:
 model= V, EV

@btime congruence(model)       #0,041 μs

  27.624 μs (192 allocations: 10.56 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 1], [1, 5]])

# FUNZIONI AGGIUNTIVE

### RemoveIntersection

rimuove le intersezioni contenute in **covers** che i boundingbox hanno con se stessi



In [32]:
function removeIntersection(covers::Array{Array{Int64,1},1})
    @threads for k=1:length(covers)
        covers[k] = setdiff(covers[k],[k])	#toglie le intersezioni con se stesso 
    end
end

removeIntersection (generic function with 1 method)

In [33]:
@btime removeIntersection(cov)    # 7.011 μs

  7.016 μs (46 allocations: 3.36 KiB)


### CreateIntervalTree

funzione che crea un albero di supporto per la funzione principale **boxcovering**; nel particolare dato un dizionario ordinato crea un intervalTrees cioè una struttura dati che contiene intervalli e che consente di trovare in modo efficiente tutti gli intervalli che si sovrappongono a un determinato intervallo o punto.

In [24]:
function createIntervalTree(boxdict::AbstractDict{Array{Float64,1},Array{Int64,1}})
    tree = IntervalTrees.IntervalMap{Float64,Array}()
    for (key, boxset) in boxdict
        tree[tuple(key...)] = boxset
    end
    return tree
end

createIntervalTree (generic function with 1 method)

In [25]:
t=createIntervalTree(dict)  #creazione dell'albero t

IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]
(1.0,1.0) => [2]

In [143]:
@btime createIntervalTree(dict)    #1,251 ns

  1.251 μs (16 allocations: 3.00 KiB)


IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]
(1.0,1.0) => [2]

createIntervalTree è type stable

In [221]:
# benchmak della funzione appena creta 'createIntervalTree'. Restituisce le velcità massime, minime e medie 
# per il calcolo dell'output.
@benchmark createIntervalTree(dict)   #min = 1,233 ns, mean = 1.320 μs, max = 267.192 μs

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.266 μs … 451.529 μs  ┊ GC (min … max):  0.00% … 99.32%
 Time  (median):     1.350 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   1.802 μs ±   9.606 μs  ┊ GC (mean ± σ):  12.46% ±  2.43%

  ▇█▆▅▅▃▂            ▁▁                                       ▁
  █████████▇▇▆▆▅▆▅▆▇████▇▇▆▆▆▅▆▅▄▅▅▄▄▄▅▅▃▅▄▅▄▄▄▂▄▄▅▄▄▄▅▄▄▄▄▄▅ █
  1.27 μs      Histogram: log(frequency) by time      4.48 μs <

 Memory estimate: 3.00 KiB, allocs estimate: 16.

### addIntersection

addIntersection aggiunge in 'covers' in i-esima posizione tutti i bounding box che intersecano l'i-esimo bounding box



In [22]:
function addIntersection(covers::Array{Array{Int64,1},1}, i::Int64, iterator)
    splice!(covers[i],1)		#splice serve a togliere gli zeri iniziali all'interno di covers
    @threads for x in collect(iterator)
        append!(covers[i],x.value)
    end
end

addIntersection (generic function with 1 method)